In [1]:

#!unzip anime-faces
!pip install tqdm
#Import everything that is needed from Keras library.
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
import glob
#matplotlib will help with displaying the results
import matplotlib.pyplot as plt
#numpy for some mathematical operations
import numpy as np
#PIL for opening,resizing and saving images
from PIL import Image
#tqdm for a progress bar when loading the dataset
from tqdm import tqdm
import scipy as sp
#os library is needed for extracting filenames from the dataset folder.
import os

def scale_images(images, new_shape):
    images_list = list()
    for image in images:
        # resize with nearest neighbor interpolation
        new_image = np.resize(image, new_shape)
        # store
        images_list.append(new_image)
    return np.asarray(images_list)
def get_fid(act1, act2):
    # calculate mean and covariance statistics
    mu1, sigma1 = act1.mean(axis=0), np.cov(act1, rowvar=False)
    mu2, sigma2 = act2.mean(axis=0), np.cov(act2, rowvar=False)
    # calculate sum squared difference between means
    ssdiff = np.sum((mu1 - mu2)**2.0)
    # calculate sqrt of product between cov
    covmean = sp.linalg.sqrtm(sigma1.dot(sigma2))
    # check and correct imaginary numbers from sqrt
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    # calculate score
    fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    return fid


class FaceGenerator:
    #RGB-images: 3-channels, grayscale: 1-channel, RGBA-images: 4-channels
    def __init__(self,image_width,image_height,channels):
        self.image_width = image_width
        self.image_height = image_height

        self.channels = channels

        self.image_shape = (self.image_width,self.image_height,self.channels)

        #Amount of randomly generated numbers for the first layer of the generator.
        self.random_noise_dimension = 100

        #Just 10 times higher learning rate would result in generator loss being stuck at 0.
        optimizer = Adam(0.0002,0.5)

        self.discriminator = self.build_discriminator()
        self.discriminator.compile(loss="binary_crossentropy",optimizer=optimizer,metrics=["accuracy"])
        self.generator = self.build_generator()

        #A placeholder for the generator input.
        random_input = Input(shape=(self.random_noise_dimension,))

        #Generator generates images from random noise.
        generated_image = self.generator(random_input)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        #Discriminator attempts to determine if image is real or generated
        validity = self.discriminator(generated_image)

        #Combined model = generator and discriminator combined.
        #1. Takes random noise as an input.
        #2. Generates an image.
        #3. Attempts to determine if image is real or generated.
        self.combined = Model(random_input,validity)
        self.combined.compile(loss="binary_crossentropy",optimizer=optimizer)

    def get_training_data(self,datafolder):
        print("Loading training data...")

        training_data = []
        #Finds all files in datafolder

        image_list = []
        for filename in glob.glob('./data/*.png'): #assuming gif
              
              im=Image.open(filename)
              image = Image.open(filename).convert('RGB')
              #Resizes to a desired size.
              image = image.resize((self.image_width,self.image_height),Image.ANTIALIAS)
              #Creates an array of pixel values from the image.
              pixel_array = np.asarray(image)

              training_data.append(pixel_array)
       
        
        
        #training_data is converted to a numpy array
        training_data = np.reshape(training_data,(-1,self.image_width,self.image_height,self.channels))
        return training_data


    def build_generator(self):
        #Generator attempts to fool discriminator by generating new images.
        model = Sequential()

        model.add(Dense(256*4*4,activation="relu",input_dim=self.random_noise_dimension))
        model.add(Reshape((4,4,256)))

        #Four layers of upsampling, convolution, batch normalization and activation.
        # 1. Upsampling: Input data is repeated. Default is (2,2). In that case a 4x4x256 array becomes an 8x8x256 array.
        # 2. Convolution: If you are not familiar, you should watch this video: https://www.youtube.com/watch?v=FTr3n7uBIuE
        # 3. Normalization normalizes outputs from convolution.
        # 4. Relu activation:  f(x) = max(0,x). If x < 0, then f(x) = 0.


        model.add(UpSampling2D())
        model.add(Conv2D(256,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(UpSampling2D())
        model.add(Conv2D(256,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(UpSampling2D())
        model.add(Conv2D(128,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))

        model.add(UpSampling2D())
        model.add(Conv2D(128,kernel_size=3,padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))


        # Last convolutional layer outputs as many featuremaps as channels in the final image.
        model.add(Conv2D(self.channels,kernel_size=3,padding="same"))
        # tanh maps everything to a range between -1 and 1.
        model.add(Activation("tanh"))

        # show the summary of the model architecture
        model.summary()

        # Placeholder for the random noise input
        input = Input(shape=(self.random_noise_dimension,))
        #Model output
        generated_image = model(input)

        #Change the model type from Sequential to Model (functional API) More at: https://keras.io/models/model/.
        return Model(input,generated_image)


    def build_discriminator(self):
        #Discriminator attempts to classify real and generated images
        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.image_shape, padding="same"))
        #Leaky relu is similar to usual relu. If x < 0 then f(x) = x * alpha, otherwise f(x) = x.
        model.add(LeakyReLU(alpha=0.2))

        #Dropout blocks some connections randomly. This help the model to generalize better.
        #0.25 means that every connection has a 25% chance of being blocked.
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        #Zero padding adds additional rows and columns to the image. Those rows and columns are made of zeros.
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        model.add(Conv2D(512, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))

        model.add(Dropout(0.25))
        #Flatten layer flattens the output of the previous layer to a single dimension.
        model.add(Flatten())
        #Outputs a value between 0 and 1 that predicts whether image is real or generated. 0 = generated, 1 = real.
        model.add(Dense(1, activation='sigmoid'))

        model.summary()

        input_image = Input(shape=self.image_shape)

        #Model output given an image.
        validity = model(input_image)

        return Model(input_image, validity)

    def train(self, datafolder ,epochs,batch_size,save_images_interval):
        #Get the real images
        training_data = self.get_training_data(datafolder)

        #Map all values to a range between -1 and 1.
        training_data = training_data / 127.5 - 1.

        #Two arrays of labels. Labels for real images: [1,1,1 ... 1,1,1], labels for generated images: [0,0,0 ... 0,0,0]
        labels_for_real_images = np.ones((batch_size,1))
        labels_for_generated_images = np.zeros((batch_size,1))
        mfid=[]
        for epoch in range(epochs):
            # Select a random half of images
            
            indices = np.random.randint(0,len(training_data),batch_size)
            real_images = training_data[indices]

            #Generate random noise for a whole batch.
            random_noise = np.random.normal(0,1,(batch_size,self.random_noise_dimension))
            #Generate a batch of new images.
            generated_images = self.generator.predict(random_noise)

            #Train the discriminator on real images.
            discriminator_loss_real = self.discriminator.train_on_batch(real_images,labels_for_real_images)
            #Train the discriminator on generated images.
            discriminator_loss_generated = self.discriminator.train_on_batch(generated_images,labels_for_generated_images)
            #Calculate the average discriminator loss.
            discriminator_loss = 0.5 * np.add(discriminator_loss_real,discriminator_loss_generated)

            #Train the generator using the combined model. Generator tries to trick discriminator into mistaking generated images as real.
            generator_loss = self.combined.train_on_batch(random_noise,labels_for_real_images)
            print ("%d [Discriminator loss: %f, acc.: %.2f%%] [Generator loss: %f]" % (epoch, discriminator_loss[0], 100*discriminator_loss[1], generator_loss))
            images1 = scale_images(real_images, (16,16,3))
            images2 = scale_images(generated_images, (16,16,3))
            act1 = images1.reshape(batch_size,768  )
            act2 = images2.reshape(batch_size,768  )
            temp=(get_fid(act1, act2))
            print('fid : ' + str(temp))
    
            mfid.append(temp)

            if epoch % save_images_interval == 0:
                self.save_images(epoch)

        #Save the model for a later use
        #self.generator.save("saved_models/facegenerator.h5")
        for i in range(0,5000):
            print(mfid[i])


    def save_images(self,epoch):
        #Save 25 generated images for demonstration purposes using matplotlib.pyplot.
        rows, columns = 5, 5
        noise = np.random.normal(0, 1, (rows * columns, self.random_noise_dimension))
        generated_images = self.generator.predict(noise)

        generated_images = 0.5 * generated_images + 0.5

        figure, axis = plt.subplots(rows, columns)
        image_count = 0
        for row in range(rows):
            for column in range(columns):
                axis[row,column].imshow(generated_images[image_count, :], cmap='spring')
                axis[row,column].axis('off')
                image_count += 1
        figure.savefig('./Imagenet/anime/generated_' + str(epoch)+ '.png')
        #figure.show()
        plt.close()

    def generate_single_image(self,model_path,image_save_path):
        noise = np.random.normal(0,1,(1,self.random_noise_dimension))
        model = load_model(model_path)
        generated_image = model.predict(noise)
        #Normalized (-1 to 1) pixel values to the real (0 to 256) pixel values.
        generated_image = (generated_image+1)*127.5
        print(generated_image)
        #Drop the batch dimension. From (1,w,h,c) to (w,h,c)
        generated_image = np.reshape(generated_image,self.image_shape)

        image = Image.fromarray(generated_image,"RGB")
        #image.save(image_save_path)

if __name__ == '__main__':
    facegenerator = FaceGenerator(64,64,3)
    facegenerator.train(datafolder="test",epochs=5000, batch_size=32, save_images_interval=100)
    #facegenerator.generate_single_image("saved_models/facegenerator.h5","test.png")
    

You should consider upgrading via the 'pip install --upgrade pip' command.


Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 17, 17, 64)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 17, 17, 64)        256       
_______________

Loading training data...


/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


...

fid : 146.94078378009448
4793 [Discriminator loss: 0.259643, acc.: 84.38%] [Generator loss: 3.993994]
fid : 124.52743081716223
4794 [Discriminator loss: 0.175949, acc.: 90.62%] [Generator loss: 3.437783]
fid : 130.97406310932715
4795 [Discriminator loss: 0.768622, acc.: 65.62%] [Generator loss: 3.042436]
fid : 130.4935870915719
4796 [Discriminator loss: 0.289569, acc.: 90.62%] [Generator loss: 4.446529]
fid : 133.27775971248096
4797 [Discriminator loss: 0.074366, acc.: 96.88%] [Generator loss: 4.841086]
fid : 103.62907213666335
4798 [Discriminator loss: 0.204054, acc.: 92.19%] [Generator loss: 3.832239]
fid : 157.8700006965916
4799 [Discriminator loss: 0.391210, acc.: 85.94%] [Generator loss: 3.621665]
fid : 138.5367189231942
4800 [Discriminator loss: 0.246308, acc.: 90.62%] [Generator loss: 4.730722]
fid : 171.03706692064588
4801 [Discriminator loss: 0.598981, acc.: 75.00%] [Generator loss: 4.121684]
fid : 138.6310199653731
4802 [Discriminator loss: 0.749317, acc.: 67.19%] [Generator 

fid : 191.13883398453865
4874 [Discriminator loss: 0.193743, acc.: 90.62%] [Generator loss: 3.073164]
fid : 153.55928673605376
4875 [Discriminator loss: 0.570340, acc.: 71.88%] [Generator loss: 2.965319]
fid : 134.1843639294211
4876 [Discriminator loss: 0.586783, acc.: 73.44%] [Generator loss: 3.040208]
fid : 172.93220660974276
4877 [Discriminator loss: 0.599242, acc.: 78.12%] [Generator loss: 3.374701]
fid : 197.94077877507658
4878 [Discriminator loss: 0.322225, acc.: 82.81%] [Generator loss: 5.502294]
fid : 165.5618667770147
4879 [Discriminator loss: 0.613105, acc.: 75.00%] [Generator loss: 3.891438]
fid : 163.5156623879459
4880 [Discriminator loss: 0.362222, acc.: 82.81%] [Generator loss: 3.715692]
fid : 213.20138279052406
4881 [Discriminator loss: 0.138518, acc.: 98.44%] [Generator loss: 3.788469]
fid : 195.79682642612102
4882 [Discriminator loss: 0.496101, acc.: 73.44%] [Generator loss: 4.703979]
fid : 196.696194762252
4883 [Discriminator loss: 0.290677, acc.: 89.06%] [Generator l

fid : 121.48789328607359
4955 [Discriminator loss: 0.277512, acc.: 87.50%] [Generator loss: 5.202878]
fid : 107.92469612779448
4956 [Discriminator loss: 0.856213, acc.: 67.19%] [Generator loss: 3.291922]
fid : 126.8453188593607
4957 [Discriminator loss: 0.358217, acc.: 89.06%] [Generator loss: 5.704382]
fid : 206.21096756755608
4958 [Discriminator loss: 0.883562, acc.: 64.06%] [Generator loss: 3.806170]
fid : 118.77192821945462
4959 [Discriminator loss: 0.289863, acc.: 84.38%] [Generator loss: 2.897097]
fid : 138.80087796381954
4960 [Discriminator loss: 0.212441, acc.: 92.19%] [Generator loss: 3.517385]
fid : 108.49216151269442
4961 [Discriminator loss: 0.226579, acc.: 92.19%] [Generator loss: 2.471974]
fid : 124.26903458504245
4962 [Discriminator loss: 0.559437, acc.: 79.69%] [Generator loss: 5.416798]
fid : 118.88477233569644
4963 [Discriminator loss: 0.266797, acc.: 87.50%] [Generator loss: 4.293242]
fid : 114.03586166846917
4964 [Discriminator loss: 1.099577, acc.: 50.00%] [Generat

117.76714792278739
107.2337367326869
132.23267981239917
138.73213179486578
107.688405258225
92.09738888549819
123.13395304919828
114.05896619323006
122.535900688004
113.74281257820142
104.64184905484387
152.06504670986416
123.81941813535224
135.45536496036482
129.4776292358149
147.58954229831886
131.36990951566227
145.35383489770868
128.0243104461021
148.16209389662492
132.6186259260268
138.79605044348057
125.41623704788259
146.87956248660174
152.1639317192813
176.06973121464608
159.76121132312028
143.42340415896103
163.38085391904252
148.34150245468797
138.53135500384002
125.07887985436174
111.04403923740014
98.77998829265323
108.20797804103981
156.60563893240385
120.94196495115423
141.25000106691797
183.8377235614064
138.59325924462598
131.4153471621236
113.44111571661456
110.30497968248159
109.54016955366568
110.86180152470183
145.9990919429634
141.99770507488444
140.33999428940427
149.21319721827678
145.74980567746013
123.03768770122609
152.28663743020724
165.34815755429008
165.700

136.71318101543142
146.10869107373486
178.03757465720054
163.5725541158211
179.8263019069164
133.97802862865564
130.18061138226318
154.14873672301394
204.0577125784807
138.21312008202509
142.59947677755596
138.0873451598804
137.44025446913432
122.63073644058264
120.47121893336237
146.21450696963214
166.66786312252387
118.83320174921803
156.8078234791955
154.39305364827902
107.97069117327362
136.4424396211699
289.79303369578287
208.32286840396935
136.5316298905783
146.53190396644152
111.38286449825085
122.97828725185198
111.90626596235991
126.39816736859264
122.18405973480336
132.27960659453785
118.33990512804795
134.33616832302042
139.30085780481733
157.40988302009916
154.63872000338066
131.313271220831
102.92316770727072
104.340002770865
173.6584448014719
98.06708422261626
134.88858855806149
113.62880112434057
98.95888862618571
120.09958028778867
124.24964592452466
117.23459028867379
88.69961398169181
99.64174800020372
92.52295444276375
91.28420714896893
114.44079013048662
91.50028272